In [1]:
import pandas as pd
import numpy as np
import scanpy as sc


In [37]:
chembldir = '/n/groups/marks/projects/scperturb/drug_norm_compcancer_paper/'

datadir = '/n/groups/marks/projects/scperturb/datasets/'

In [20]:

#: no match whatsoever could be found
no_match = pd.read_csv(chembldir+'missing_normalized_drug_names_NONE.csv', index_col=0 )
### READY:

# assign ChEMBL ID directly
exact_match = pd.read_csv(chembldir+'normalized_drug_names_EXACT.csv' , index_col=0)

#With a little processing I could assign a ChEMBL ID : 
# these entries were in the form `NAME (CODE_NAME_1, CODE_NAME_2, ...
substring_match = pd.read_csv(chembldir+'normalized_drug_names_SUBSTRING.csv', index_col=0 )

# partial match, due to typo, add manually by searching amended name in ChEMBL web page
manual_partial_match = pd.read_csv(chembldir+'manual_normalized_drug_names_PARTIAL.csv', index_col=0)
#- : no match, but was able to ID manually
manual_match = pd.read_csv(chembldir+'manual_normalized_drug_names_NONE.csv', index_col=0) 

In [17]:
substring_match.columns

Index(['Name', 'Dataset', 'Found Names', 'Found Name IDs', 'Match Type',
       'Edit Distance', 'Query Time', 'ChEMBL', 'Manual normalization'],
      dtype='object')

In [18]:
manual_partial_match.columns

Index(['Name', 'Dataset', 'Found Names', 'Found Name IDs', 'Match Type',
       'Edit Distance', 'Query Time', 'Manual normalization', 'ChEMBL'],
      dtype='object')

In [78]:
drugs = pd.concat([no_match, exact_match, substring_match,manual_partial_match, manual_match])

In [79]:
dsets = drgs['Dataset'].value_counts().index.values

In [80]:
dset  = 'SrivatsanTrapnell2020_sciplex3'

In [102]:
def add_chembl(dset, drugs=drugs, datadir=datadir):
    #obs = pd.read_csv(datadir+dset+'/obs.csv')

    drg = drugs[drugs['Dataset'] == dset]
    drugdict = dict(zip(drg['Name'],drg['ChEMBL']))
    if dset == "McFarlandTshemiak2020_all_expts_combined":
        dset = "McFarlandTsherniak2020"
    adata  = sc.read_h5ad(datadir+dset+'/'+dset+'.h5ad')
    adata.obs['chembl-ID'] = adata.obs['perturbation'].map(drugdict)
    # write csv obs
    adata.obs.to_csv(datadir+dset+'/obs.csv')
    # write adata WITH GZIP
    adata.write_h5ad(datadir+dset+'/'+dset+'.h5ad', compression = 'gzip')
    print(dset+'_done')
    return(adata.obs)

In [103]:
for dset in dsets[1:]: add_chembl(dset)

McFarlandTsherniak2020_done
SrivatsanTrapnell2020_sciplex4_done
ZhaoSims2021_done
SrivatsanTrapnell2020_sciplex2_done
ChangYe2021_done
AissaBenevolenskaya2021_done
GehringPachter2019_done


In [96]:
dset

'McFarlandTshemiak2020_all_expts_combined'